In [1]:
import requests
from bs4 import BeautifulSoup
import pandas

In [3]:
def findGroupPageNumber(groupID):
    startURL = "https://www.douban.com/group/" + str(groupID) + "/members?start=0"
    r = requests.get(startURL, headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'})
    c = r.content
    soup = BeautifulSoup(c, "html.parser")
    pageNumber = int(soup.find("span", {"class":"thispage"}).attrs["data-total-page"])
    return pageNumber

In [4]:
def doubanGroupMemberSpider(groupID):
    result = []
    baseURL = "https://www.douban.com/group/" + str(groupID) + "/members?start="
    pageNumber = findGroupPageNumber(groupID)
    for page in range(0, pageNumber*36, 36):
        currentPageURL = baseURL + str(page)
        r = requests.get(currentPageURL, headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'})
        c = r.content
        soup = BeautifulSoup(c, "html.parser")
        all = soup.find_all("div", {"class":"name"})
        for item in all:
            info = {}
            # Username 用户名
            info["UserName"] = item.find("a").text
            # Location 用户地点
            try:
                info["Location"] = item.find("span", {"class":"pl"}).text[1:-1]
            except:
                info["Location"] = None
            # User Home Page URL 用户主页网址
            userHomeURL = item.find('a').attrs['href']
            info["UserID"] = userHomeURL.replace("https://www.douban.com/people", "")[1:-1]
            info["PersonalHomePgae"] = userHomeURL
            result.append(info)
    df = pandas.DataFrame(result)
    return df

In [5]:
# Check whether a user is in this group
def isIN(groupID, inputUserID):
    groupInfo = doubanGroupMemberSpider(groupID)
    groupIDs = groupInfo["UserID"].values
    return inputUserID in groupIDs

In [7]:
groupID = 698362
Group698362 = doubanGroupMemberSpider(groupID)
Group698362.tail()

,UserName,Location,UserID,PersonalHomePgae
25057,随心,青岛,208889978,https://www.douban.com/people/208889978/
25058,已注销,上海,nuts93,https://www.douban.com/people/nuts93/
25059,爱吃饭的镁,,181577509,https://www.douban.com/people/181577509/
25060,🍒,,179741179,https://www.douban.com/people/179741179/
25061,做你说过的,,181266328,https://www.douban.com/people/181266328/
